In [40]:
import numpy as np


def composite(ifrom, ito, values, comp_length=1.0, min_comp_length=-1.0):
    """
    cfrom,cto,clen,cvar,cacum= composite(ifrom, ito, values,
                                        comp_length = 1, min_comp_length=-1)

    Composite intervals in a single drillhole. The From-To
    intervals may be sorted.

    Parameters
    ----------
    ifrom, ito:     1D arrays of floats
        From - To  interval
    values :   1D array of floats/integers
        variable,
    comp_length: Optional, float, default 1.
        length of the compositing intervals
    min_comp_length: Optional, float, defaul -1.
        minimum length of the composite, if <=0 then min_comp_length = comp_length/2.

    Return
    -------
    (cfrom, cto, clen, cvar, cacum)
    cfrom, cto:  1D arrays of floats
         From, To composited intervals
    clen, cvar, cacum:  1D arrays of floats
         total length of intervals composited
         variable composited
         variable accumulated

    """

    assert ifrom.shape == ito.shape, "Error: ifrom and ito with different shape"
    assert all(ifrom < ito), "Error: ifrom >= ito, wrong or zero length intervals"
    assert all(np.isfinite(ifrom)), "Error: ifrom with not finite elements"
    assert all(np.isfinite(ito)), "Error: ito with not finite elements"

    if min_comp_length <= 0:
        min_comp_length = comp_length / 2.0

    ncomp = int(ito[-1] / comp_length + 1)
    nintrb = len(ifrom)

    # create the composite arrays
    cfrom = np.arange(0.0, ito[-1] + comp_length, comp_length)
    cto = cfrom + comp_length
    clen = np.zeros(cto.shape)
    cvar = np.zeros(cto.shape)
    cvar[:] = np.nan
    cacum = np.zeros(cto.shape)

    iprop = np.zeros(ito.shape)

    # for each composite
    for i in range(ncomp):

        # initialize proportions
        iprop[:] = 0

        # for each interval
        for l in range(nintrb):

            # ignore interval if variable is nan
            if np.isnan(values[l]):
                continue

            # case a, below the composite
            if ifrom[l] >= cto[i]:
                break

            # case b, over the composite
            if ito[l] <= cfrom[i]:
                continue

            # --these are overlap--

            # case overlap top or contained
            if ito[l] > cfrom[i] and ito[l] <= cto[i]:

                # case c, interval in composite
                if ifrom[l] >= cfrom[i]:
                    iprop[l] = ito[l] - ifrom[l]

                # case d, overlap top
                else:
                    iprop[l] = ito[l] - cfrom[i]

            # case e, composite in interval
            if ifrom[l] < cfrom[i] and ito[l] > cto[i]:
                iprop[l] = cto[i] - cfrom[i]
                continue

            # case f, overlap bottom
            if ifrom[l] >= cfrom[i] and ifrom[l] < cto[i] and ito[l] > cto[i]:
                iprop[l] = cto[i] - ifrom[l]
                continue

        clen[i] = np.nansum(iprop)

        if clen[i] > min_comp_length:
            cacum[i] = np.nansum(values * iprop)
            cvar[i] = cacum[i] / clen[i]  # wighted average
        else:
            cvar[i] = np.nan
            cacum[i] = np.nan

    return cfrom, cto, clen, cvar, cacum

In [3]:
import pandas as pd

In [4]:
assay = pd.read_csv("../data/assay.csv")

In [5]:
assay.head()

,DHID,FROM,TO,_len,DUM,_acum,azmm,dipm,xm,ym,...,azme,dipe,xe,ye,ze,dist_hw,dist_fw,D1_surf,D1_solid,Au
0,0,0.0,1.0,1.0,0.0,0.0,90.0,86.774025,4.561190,-1.229899e-09,...,90.0,86.774147,4.589327,-2.459786e-09,98.728762,-1.0,-1.0,-1.0,0,0.0
1,0,1.0,2.0,1.0,0.0,0.0,90.0,86.774147,4.617463,-3.689650e-09,...,90.0,86.774086,4.645599,-4.919526e-09,97.730346,-1.0,-1.0,-1.0,0,0.0
2,0,2.0,3.0,1.0,0.0,0.0,90.0,86.774086,4.673736,-6.149413e-09,...,90.0,86.774025,4.701872,-7.379312e-09,96.731931,-1.0,-1.0,-1.0,0,0.0
3,0,3.0,4.0,1.0,0.0,0.0,90.0,86.774086,4.730009,-8.609211e-09,...,90.0,86.774086,4.758146,-9.839098e-09,95.733516,-1.0,-1.0,-1.0,0,0.0
4,0,4.0,5.0,1.0,0.0,0.0,90.0,86.774086,4.786282,-1.106898e-08,...,90.0,86.774086,4.814419,-1.229887e-08,94.735100,-1.0,-1.0,-1.0,0,0.0


In [8]:
dh = assay[assay.DHID==1]

In [41]:
composite(dh.FROM.values, dh.TO.values, dh.Au.values, comp_length=5)

(array([  0.,   5.,  10.,  15.,  20.,  25.,  30.,  35.,  40.,  45.,  50.,
         55.,  60.,  65.,  70.,  75.,  80.,  85.,  90.,  95., 100.]),
 array([  5.,  10.,  15.,  20.,  25.,  30.,  35.,  40.,  45.,  50.,  55.,
         60.,  65.,  70.,  75.,  80.,  85.,  90.,  95., 100., 105.]),
 array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
        5., 5., 3., 0.]),
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.20670178, 0.44581511, 0.63424236, 1.13118806,
        0.52430974, 0.88292212, 0.11915897, 0.        , 0.        ,
               nan]),
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.03350891, 2.22907555, 3.17121182, 5.65594032,
        2.62154868, 4.4146106 , 0.59579487, 0.        , 0.        ,
        0.        ]))

In [50]:
def composite_dh(dh, comp_len, var_name):
    return composite(dh.FROM.values, dh.TO.values, dh[var_name].values, comp_len)
    

In [54]:
groups = assay.groupby("DHID")
result = groups.apply(composite_dh, 5,"Au")

In [62]:
df = pd.DataFrame(list(result))

In [91]:
pd.DataFrame(result).T

DHID,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,"([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...",...,"([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0], [5....","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0...","([0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0], [5...."
